In [13]:
import numpy as np

In [2]:
from rbc.omniscidb import RemoteOmnisci

In [10]:
omnisci = RemoteOmnisci(user='admin', password='HyperInteractive',
                     host='127.0.0.1', port=6274, dbname='omnisci')

In [19]:
SQL=lambda query: np.array(list(omnisci.sql_execute(query)[1]), dtype=object)

In [14]:
SQL('DROP TABLE IF EXISTS mytable')
SQL('CREATE TABLE IF NOT EXISTS mytable (i INT[]);');
for i in range(5):
    SQL('INSERT INTO mytable VALUES (ARRAY[1,2,3,'+str(i)+'])');
list(SQL('SELECT * FROM mytable')[1])

[array([1, 2, 3, 1])]

In [17]:
@omnisci('i32(i32[])')
def sum_i32(x):
    s = 0
    for i in range(len(x)):
        s += x[i]
    return s

In [20]:
SQL('SELECT i, sum_i32(i) FROM mytable')

array([[list([1, 2, 3, 0]), 6],
       [list([1, 2, 3, 1]), 7],
       [list([1, 2, 3, 2]), 8],
       [list([1, 2, 3, 3]), 9],
       [list([1, 2, 3, 4]), 10]], dtype=object)

In [21]:
print(sum_i32)


--------------------------------------cpu---------------------------------------
; ModuleID = 'rbc.irtools.compile_to_IR'
source_filename = "<string>"
target triple = "x86_64-unknown-linux-gnu"

@"_ZN08NumbaEnv8__main__11sum_i32$247E22$28STRUCT__iPLbK$29$2a" = common local_unnamed_addr global i8* null
@"_ZN08NumbaEnv3rbc15omnisci_backend14omnisci_buffer18omnisci_buffer_len12$3clocals$3e16$3clambda$3e$248E22$28STRUCT__iPLbK$29$2a" = common local_unnamed_addr global i8* null
@"_ZN08NumbaEnv3rbc15omnisci_backend14omnisci_buffer22omnisci_buffer_getitem12$3clocals$3e16$3clambda$3e$249E22$28STRUCT__iPLbK$29$2ax" = common local_unnamed_addr global i8* null

; Function Attrs: nofree norecurse nounwind readonly
define i32 @sum_i32_ia___iPLbKPA({ i32*, i64, i8 }* nocapture readonly %.1) local_unnamed_addr #0 {
entry:
  %.17.i.i = getelementptr { i32*, i64, i8 }, { i32*, i64, i8 }* %.1, i64 0, i32 1
  %.18.i.i = load i64, i64* %.17.i.i, align 8, !noalias !0
  %.107.i = icmp sgt i64 %.18.i.i, 0
 